In [94]:
import trackhhl.toy.simple_generator as toy
import trackhhl.event_model.q_event_model as em
import numpy as np
import itertools
import copy
from dwave.samplers import SimulatedAnnealingSampler
import dimod
import psutil
import time
from scipy.sparse import lil_matrix, csc_matrix, block_diag




In [95]:
N_MODULES = 5
LX = float("+inf")
LY = float("+inf")
Z_SPACING = 1.0

detector = toy.SimpleDetectorGeometry(
    module_id=list(range(N_MODULES)),
    lx=[LX] * N_MODULES,
    ly=[LY] * N_MODULES,
    z=[i + Z_SPACING for i in range(N_MODULES)]
)

generator = toy.SimpleGenerator(
    detector_geometry=detector,
    theta_max=np.pi / 6
)

N_PARTICLES = 2
event = generator.generate_event(N_PARTICLES)
event.hits

[Hit(hit_id=0, x=0.1900580289252624, y=0.20583556882894866, z=1.0, module_id=0, track_id=0),
 Hit(hit_id=5, x=0.14332521972526407, y=0.26095267152237683, z=1.0, module_id=0, track_id=1),
 Hit(hit_id=1, x=0.3801160578505248, y=0.4116711376578973, z=2.0, module_id=1, track_id=0),
 Hit(hit_id=6, x=0.28665043945052815, y=0.5219053430447537, z=2.0, module_id=1, track_id=1),
 Hit(hit_id=2, x=0.5701740867757872, y=0.6175067064868459, z=3.0, module_id=2, track_id=0),
 Hit(hit_id=7, x=0.4299756591757922, y=0.7828580145671303, z=3.0, module_id=2, track_id=1),
 Hit(hit_id=3, x=0.7602321157010496, y=0.8233422753157946, z=4.0, module_id=3, track_id=0),
 Hit(hit_id=8, x=0.5733008789010563, y=1.0438106860895073, z=4.0, module_id=3, track_id=1),
 Hit(hit_id=4, x=0.950290144626312, y=1.029177844144743, z=5.0, module_id=4, track_id=0),
 Hit(hit_id=9, x=0.7166260986263203, y=1.304763357611884, z=5.0, module_id=4, track_id=1)]

In [97]:
def generate_hamiltonian_optimized(event, params):
    """
    Generates the Hamiltonian matrix using sparse block storage to ensure it is consistent with the non-optimized version.
    The matrix is divided into blocks that store only non-zero interactions, ensuring consistent signs.
    """
    lambda_val = params.get('lambda')
    alpha = params.get('alpha')
    beta = params.get('beta')

    # Sort modules by their z position
    modules = sorted(event.modules, key=lambda module: module.z)

    # Generate only segments between adjacent modules (layer-to-layer connections)
    segments = [
        em.segment(from_hit, to_hit)
        for idx in range(len(modules) - 1)
        for from_hit, to_hit in itertools.product(modules[idx].hits, modules[idx + 1].hits)
    ]

    N = len(segments)  # Number of segments

    # Initialize sparse block matrices (lil_matrix allows efficient row-wise construction)
    A_ang_blocks = []
    A_bif_blocks = []
    A_inh_blocks = []
    b = np.zeros(N)

    # Divide the Hamiltonian into smaller blocks
    block_size = 500  # Adjust block size based on memory availability
    num_blocks = (N + block_size - 1) // block_size  # Calculate how many blocks are needed

    for block_idx in range(num_blocks):
        start_idx = block_idx * block_size
        end_idx = min(start_idx + block_size, N)

        # Create lil_matrix for each block
        A_ang_block = lil_matrix((end_idx - start_idx, end_idx - start_idx), dtype=np.float32)
        A_bif_block = lil_matrix((end_idx - start_idx, end_idx - start_idx), dtype=np.float32)
        A_inh_block = lil_matrix((end_idx - start_idx, end_idx - start_idx), dtype=np.float32)

        # Fill the block with appropriate values
        for i in range(start_idx, end_idx):
            seg_i = segments[i]
            vect_i = seg_i.to_vect()
            norm_i = np.linalg.norm(vect_i)

            for j in range(i + 1, end_idx):  # Symmetry, only compute for i < j
                seg_j = segments[j]
                vect_j = seg_j.to_vect()
                norm_j = np.linalg.norm(vect_j)

                # Angular consistency (cosine similarity close to 1)
                cosine = np.dot(vect_i, vect_j) / (norm_i * norm_j)
                if np.abs(cosine - 1) < 1e-9:
                    A_ang_block[i - start_idx, j - start_idx] = 1
                    A_ang_block[j - start_idx, i - start_idx] = 1  # Symmetry with positive sign

                # Bifurcation penalties (Ensure correct negative sign)
                if seg_i.from_hit == seg_j.from_hit and seg_i.to_hit != seg_j.to_hit:
                    A_bif_block[i - start_idx, j - start_idx] = -alpha
                    A_bif_block[j - start_idx, i - start_idx] = -alpha  # Symmetry with negative sign

                if seg_i.from_hit != seg_j.from_hit and seg_i.to_hit == seg_j.to_hit:
                    A_bif_block[i - start_idx, j - start_idx] = -alpha
                    A_bif_block[j - start_idx, i - start_idx] = -alpha  # Symmetry with negative sign

                # Inhomogeneity penalties (Ensure positive sign)
                s_ab = int(seg_i.from_hit.module_id == 1 and seg_j.to_hit.module_id == 1)
                if s_ab > 0:
                    A_inh_block[i - start_idx, j - start_idx] = beta * s_ab * s_ab
                    A_inh_block[j - start_idx, i - start_idx] = beta * s_ab * s_ab  # Symmetry with positive sign

        # Store each block
        A_ang_blocks.append(A_ang_block)
        A_bif_blocks.append(A_bif_block)
        A_inh_blocks.append(A_inh_block)

    # Combine all blocks into the final Hamiltonian matrix using block diagonal
    A_ang = block_diag(A_ang_blocks, format='csc')
    A_bif = block_diag(A_bif_blocks, format='csc')
    A_inh = block_diag(A_inh_blocks, format='csc')

    # Combine into the full Hamiltonian, ensuring correct sign handling
    A = -1 * (A_ang + A_bif + A_inh)

    return A, b, segments
print(A.toarray())

# Performance measurement
process = psutil.Process()

start_memory = process.memory_info().rss / (1024 ** 2)  # Memory in MB
start_time = time.time()

# Call the optimized function
A, b, segments = generate_hamiltonian_optimized(event, params)

end_memory = process.memory_info().rss / (1024 ** 2)  # Memory in MB
end_time = time.time()

memory_used = end_memory - start_memory
time_taken = end_time - start_time

print(f"Memory used: {memory_used:.2f} MB")
print(f"Time taken: {time_taken:.6f} seconds")

[[ 0.  1.  1.  0. -1.  0.  0.  0. -1.  0.  0.  0. -1.  0.  0.  0.]
 [ 1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  1.  0.  0.  0.  0. -1.  0.  0.  0. -1.  0.  0.  0. -1.]
 [-1.  0.  0.  0.  0.  1.  1.  0. -1.  0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  1.  1.  0.  0.  0.  0. -1.  0.  0.  0. -1.]
 [-1.  0.  0.  0. -1.  0.  0.  0.  0.  1.  1.  0. -1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0. -1.  0.  1.  1.  0.  0.  0.  0. -1.]
 [-1.  0.  0.  0. -1.  0.  0.  0. -1.  0.  0.  0.  0.  1.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0. 